# Preprocessing notebook for Zillow Housing Data
### Data is obtained from https://www.zillow.com/research/data/
#### Included datasets:
* Zillow Home Value Index (ZHVI) - separate datasets for prices by **Country(US)/Metro**, **State**, **County**, **City**, **ZIP Code**, and **Neighborhood**
* Zillow Home Value *Forecast* (ZHVF) - by **Country(US)/Metro** and **ZIP code**
* Zillow Observed Rent Index (ZORI) - by **Country(US)/Metro**, **County**, **City**, and **ZIP code**
* For-Sale Listings - by **Country(US)/Metro**
* Sales (Estimated number of properties sold in a month) - by **Country(US)/Metro**
* Market Heat Index (higher number $\rightarrow$ market favors sellers) - by **Country(US)/Metro**
* New Construction - by **Country(US)/Metro**


In [1]:
# Mount Google Drive, data will be in /content/drive/MyDrive/Zillow_Housing_Data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

In [3]:
### 'Melt' the dataframes so that 'time' is a single attribute
def preprocess_raw(df_dict, datecol_begin):
  regions_df = pd.DataFrame() # Initialize empty dataframe for region information
  for n,i in enumerate(df_dict):
    cols = df_dict[i].columns
    df_dict[i] = pd.melt(df_dict[i], id_vars=cols[0:datecol_begin[n]], value_vars=cols[datecol_begin[n]:]) # 'Melt' the dataframe so that we have a single 'time' attribute
    df_dict[i] = df_dict[i].dropna() # Drop rows with NA
    df_dict[i].columns = list(cols[0:datecol_begin[n]]) + ['date','value'] # Rename columns
    df_dict[i]['date'] = pd.to_datetime(df_dict[i]['date']) # Datetime
    print('cont')
    # Drop region-related columns - this information will be retained in the Regions table
    region_cols = df_dict[i].columns[0:datecol_begin[n]]
    print(region_cols)
    region_df = df_dict[i][region_cols]
    regions_df = pd.concat([regions_df, region_df])
    df_dict[i] = df_dict[i].drop(columns=region_cols[1:])
  return [df_dict, regions_df]

In [4]:
### Read in Zillow Home Value Index (ZHVI) data
zhvi_metro = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowHomeValueIndex_byMetro.csv')
zhvi_state = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowHomeValueIndex_byState.csv')
zhvi_state.drop(columns=['StateName'], inplace=True)
zhvi_county = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowHomeValueIndex_byCounty.csv')
zhvi_city = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowHomeValueIndex_byCity.csv')
zhvi_zip = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowHomeValueIndex_byZIPCode.csv')
zhvi_neighborhood = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowHomeValueIndex_byNeighborhood.csv')
zhvi = {
    "zhvi_metro": zhvi_metro,
    "zhvi_state": zhvi_state,
    "zhvi_county": zhvi_county,
    "zhvi_city": zhvi_city,
    "zhvi_zip": zhvi_zip,
    "zhvi_neighborhood": zhvi_neighborhood
}

In [5]:
zhvi_metro.columns[5:]

Index(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31',
       '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31',
       ...
       '2023-12-31', '2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
       '2024-05-31', '2024-06-30', '2024-07-31', '2024-08-31', '2024-09-30'],
      dtype='object', length=297)

In [6]:
zhvi_processed_all = preprocess_raw(zhvi, [5, 4, 9, 8, 9, 9])
zhvi_processed = zhvi_processed_all[0]
zhi_regions = zhvi_processed_all[1]
zhvi_processed['zhvi_state']

cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType'], dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS'],
      dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'CountyName'],
      dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName'],
      dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName'],
      dtype='object')


,RegionID,date,value
0,9,2000-01-31,187686.834174
1,54,2000-01-31,111067.066209
2,14,2000-01-31,106914.485631
3,43,2000-01-31,152590.518413
4,47,2000-01-31,99083.388149
...,...,...,...
15142,37,2024-09-30,262706.047454
15143,3,2024-09-30,364673.503983
15144,12,2024-09-30,601519.022311
15145,58,2024-09-30,395523.163104


In [7]:
zhi_regions

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,CountyName,City
1,394913,1,"New York, NY",msa,NY,NaN,NaN,NaN,NaN,NaN,NaN
2,753899,2,"Los Angeles, CA",msa,CA,NaN,NaN,NaN,NaN,NaN,NaN
3,394463,3,"Chicago, IL",msa,IL,NaN,NaN,NaN,NaN,NaN,NaN
4,394514,4,"Dallas, TX",msa,TX,NaN,NaN,NaN,NaN,NaN,NaN
5,394692,5,"Houston, TX",msa,TX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6423511,267809,27095,Rock Springs,neighborhood,NC,NC,"Greenville, NC",NaN,NaN,Pitt County,Greenville
6423512,251134,27095,Lind,neighborhood,CO,CO,"Fort Collins, CO",NaN,NaN,Larimer County,Fort Collins
6423513,831177,27095,Sanctuary,neighborhood,FL,FL,"Tampa-St. Petersburg-Clearwater, FL",NaN,NaN,Hillsborough County,Riverview
6423514,416956,27095,Enclave at Breckinridge,neighborhood,TX,TX,"Dallas-Fort Worth-Arlington, TX",NaN,NaN,Collin County,Richardson


In [8]:
# Drop unneeded columns from ZHVI regions table
zhi_regions.drop(columns=['State', 'StateCodeFIPS', 'MunicipalCodeFIPS'], inplace = True)

In [9]:
### Read in Zillow Observed Rent Index (ZORI)
zori_metro = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowObservedRentIndex_byMetro.csv')
zori_county = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowObservedRentIndex_byCounty.csv')
zori_city = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowObservedRentIndex_byCity.csv')
zori_zip = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowObservedRentIndex_byZIPCode.csv')
zori = {
    "zori_metro": zori_metro,
    "zori_county": zori_county,
    "zori_city": zori_city,
    "zori_zip": zori_zip
}

In [10]:
zori['zori_metro']

,RegionID,SizeRank,RegionName,RegionType,StateName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,...,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30
0,102001,0,United States,country,NaN,1242.289688,1249.025207,1258.228660,1267.585683,1277.143068,...,1977.161448,1981.134048,1989.751341,2001.329357,2013.083020,2024.992818,2035.673920,2044.246569,2048.589250,2049.885652
1,394913,1,"New York, NY",msa,NY,2358.422591,2373.827063,2392.346566,2413.503238,2429.422645,...,3237.484000,3241.452938,3260.248486,3292.313370,3321.640021,3355.901441,3384.713534,3402.829227,3408.143526,3397.349448
2,753899,2,"Los Angeles, CA",msa,CA,1838.010077,1850.538514,1866.331712,1879.953935,1894.660420,...,2887.191192,2891.927500,2904.672694,2921.463850,2931.663944,2946.476179,2959.899652,2973.461372,2979.695642,2983.960991
3,394463,3,"Chicago, IL",msa,IL,1399.429182,1406.244920,1416.037514,1425.008799,1435.471185,...,1938.487883,1948.287686,1962.821441,1977.033982,1997.523960,2016.067983,2033.575456,2042.615093,2045.885100,2044.046378
4,394514,4,"Dallas, TX",msa,TX,1116.157758,1121.285115,1128.945188,1141.081647,1151.039110,...,1767.791291,1764.425172,1761.268413,1765.506605,1770.245618,1778.210210,1784.361840,1790.635871,1793.692760,1789.640286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,394687,875,"Hood River, OR",msa,OR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2229.104452,2232.710257,2263.529384,2222.604684,2289.863204,2302.916667
638,394718,892,"Jamestown, ND",msa,ND,NaN,NaN,NaN,NaN,NaN,...,1170.411917,1149.711746,1139.198113,1142.828239,1161.187292,1174.721486,1185.294833,1205.991662,1218.605723,1245.111111
639,394996,915,"Portales, NM",msa,NM,NaN,NaN,NaN,NaN,NaN,...,949.237076,958.978329,936.642975,947.378489,955.854033,974.060784,973.852517,977.987145,976.851696,978.255952
640,394805,916,"Los Alamos, NM",msa,NM,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2347.761105,2300.000000


In [11]:
zori_processed_all = preprocess_raw(zori, [5, 9, 8, 9])
zori_processed = zori_processed_all[0]
zori_regions = zori_processed_all[1]

cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS'],
      dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'CountyName'],
      dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName'],
      dtype='object')


In [12]:
zori_processed['zori_metro']

,RegionID,date,value
1,394913,2015-01-31,2358.422591
2,753899,2015-01-31,1838.010077
3,394463,2015-01-31,1399.429182
4,394514,2015-01-31,1116.157758
5,394692,2015-01-31,1247.859047
...,...,...,...
75109,394687,2024-09-30,2302.916667
75110,394718,2024-09-30,1245.111111
75111,394996,2024-09-30,978.255952
75112,394805,2024-09-30,2300.000000


In [13]:
zori_regions

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,CountyName,City
1,394913,1,"New York, NY",msa,NY,NaN,NaN,NaN,NaN,NaN,NaN
2,753899,2,"Los Angeles, CA",msa,CA,NaN,NaN,NaN,NaN,NaN,NaN
3,394463,3,"Chicago, IL",msa,IL,NaN,NaN,NaN,NaN,NaN,NaN
4,394514,4,"Dallas, TX",msa,TX,NaN,NaN,NaN,NaN,NaN,NaN
5,394692,5,"Houston, TX",msa,TX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
834673,418163,30158,89158,zip,NV,NV,"Las Vegas-Henderson-Paradise, NV",NaN,NaN,Clark County,Las Vegas
834674,91179,30490,76005,zip,TX,TX,"Dallas-Fort Worth-Arlington, TX",NaN,NaN,Tarrant County,Arlington
834675,61618,30490,10004,zip,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",NaN,NaN,New York County,New York
834676,97071,30490,92878,zip,CA,CA,"Riverside-San Bernardino-Ontario, CA",NaN,NaN,Riverside County,Corona


In [14]:
zori_regions.drop(columns=['State','StateCodeFIPS', 'MunicipalCodeFIPS'], inplace = True)

In [15]:
zori_regions

,RegionID,SizeRank,RegionName,RegionType,StateName,Metro,CountyName,City
1,394913,1,"New York, NY",msa,NY,NaN,NaN,NaN
2,753899,2,"Los Angeles, CA",msa,CA,NaN,NaN,NaN
3,394463,3,"Chicago, IL",msa,IL,NaN,NaN,NaN
4,394514,4,"Dallas, TX",msa,TX,NaN,NaN,NaN
5,394692,5,"Houston, TX",msa,TX,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
834673,418163,30158,89158,zip,NV,"Las Vegas-Henderson-Paradise, NV",Clark County,Las Vegas
834674,91179,30490,76005,zip,TX,"Dallas-Fort Worth-Arlington, TX",Tarrant County,Arlington
834675,61618,30490,10004,zip,NY,"New York-Newark-Jersey City, NY-NJ-PA",New York County,New York
834676,97071,30490,92878,zip,CA,"Riverside-San Bernardino-Ontario, CA",Riverside County,Corona


In [27]:
### Read in Zillow Home Value Forecast (ZHVF). This doesn't need to be preprocessed because the dates are in a fine format, [end of month, end of quarter, +1 year]
zhvf_metro = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowHomeValueForecast_byMetro.csv')
zhvf_metro_regions = zhvf_metro[zhvf_metro.columns[:5]]
zhvf_metro = zhvf_metro.drop(columns=zhvf_metro_regions.columns[1:])
zhvf_metro.columns = ['RegionID', 'BaseDate', 'Month', 'Quarter', 'Year']
zhvf_zip = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ZillowHomeValueForecast_byZIPCode.csv')
zhvf_zip_regions = zhvf_zip[zhvf_zip.columns[:9]]
zhvf_zip = zhvf_zip.drop(columns=zhvf_zip_regions.columns[1:])
zhvf_zip.columns = ['RegionID', 'BaseDate', 'Month', 'Quarter', 'Year']

zhvf_regions = pd.concat([zhvf_metro_regions, zhvf_zip_regions])
zhvf = {
    "zhvf_metro": zhvf_metro,
    "zhvf_zip": zhvf_zip
}

In [61]:
# @title
zhvf_metro

,RegionID,SizeRank,RegionName,RegionType,StateName,BaseDate,2024-10-31,2024-12-31,2025-09-30
0,102001,0,United States,country,NaN,2024-09-30,0.1,0.2,1.1
1,394913,1,"New York, NY",msa,NY,2024-09-30,0.3,0.0,-0.4
2,753899,2,"Los Angeles, CA",msa,CA,2024-09-30,0.3,0.3,0.6
3,394463,3,"Chicago, IL",msa,IL,2024-09-30,0.1,-0.3,-0.8
4,394514,4,"Dallas, TX",msa,TX,2024-09-30,-0.2,-1.1,-0.5
...,...,...,...,...,...,...,...,...,...
890,753929,935,"Zapata, TX",msa,TX,2024-09-30,-0.7,-2.9,-8.7
891,394743,936,"Ketchikan, AK",msa,AK,2024-09-30,0.3,-0.3,-2.7
892,753874,937,"Craig, CO",msa,CO,2024-09-30,0.4,0.3,2.2
893,395188,938,"Vernon, TX",msa,TX,2024-09-30,-1.5,-3.0,-5.9


In [30]:
zhvf_zip

,RegionID,BaseDate,Month,Quarter,Year
0,91982,2024-09-30,0.1,-0.5,-1.1
1,61148,2024-09-30,0.5,0.1,-0.4
2,91940,2024-09-30,-0.2,-1.1,-1.2
3,62080,2024-09-30,-0.2,-0.9,-0.5
4,91733,2024-09-30,-0.2,-0.9,-1.2
...,...,...,...,...,...
20157,71617,2024-09-30,-1.2,-1.1,-0.1
20158,72865,2024-09-30,-0.1,-0.7,0.5
20159,73017,2024-09-30,-0.9,-2.3,0.2
20160,79426,2024-09-30,0.3,0.5,0.2


In [34]:
### Read in For Sale listings
forSaleListings = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/ForSaleListings_byMetro.csv')
### Read in Market Heat Index
mhi = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/MarketHeatIndex_byMetro.csv')
### Read in New Construction Sales
new_conSales = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/NewConstructionSales_byMetro.csv')
### Read in Sales
sales = pd.read_csv('/content/drive/MyDrive/Zillow_Housing_Data/Sales_byMetro.csv')

### Process remaining data
forSaleListings_processed_all = preprocess_raw({'metro':forSaleListings}, [5])
forSaleListings_processed = forSaleListings_processed_all[0]
forSaleListings_regions = forSaleListings_processed_all[1]

mhi_processed_all = preprocess_raw({'metro':mhi}, [5])
mhi_processed = mhi_processed_all[0]
mhi_regions = mhi_processed_all[1]

newConSales_processed_all = preprocess_raw({'metro':new_conSales}, [5])
newConSales_processed = newConSales_processed_all[0]
newConSales_regions = newConSales_processed_all[1]

sales_processed_all = preprocess_raw({'metro':sales}, [5])
sales_processed = sales_processed_all[0]
sales_regions = sales_processed_all[1]

cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], dtype='object')
cont
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], dtype='object')


In [58]:
### Make final regions table
regions = pd.concat([zhi_regions, zori_regions, zhvf_regions, forSaleListings_regions, mhi_regions, newConSales_regions, sales_regions])
regions = regions.drop_duplicates(subset=['RegionID'])

In [56]:
np.unique(regions.RegionType)

array(['city', 'country', 'county', 'msa', 'neighborhood', 'state', 'zip'],
      dtype=object)

In [59]:
regions.to_csv('/content/drive/MyDrive/Zillow_Housing_Data/Regions_cleaned.csv')

In [57]:
### Write data
regions.to_csv('/content/drive/MyDrive/Zillow_Housing_Data/Regions_cleaned.csv')
all_processed = {'zhvi_processed':zhvi_processed, 'zori_processed':zori_processed, 'zhvf':zhvf, 'forSaleListings_processed':forSaleListings_processed, 'mhi_processed':mhi_processed, 'newConSales_processed':newConSales_processed, 'sales_processed':sales_processed}
for d in all_processed:
  curr = all_processed[d]
  for k in curr:
    f = '/content/drive/MyDrive/Zillow_Housing_Data/{}_by-{}_cleaned.csv'.format(d,k)
    print(f)
    curr[k].to_csv(f)

/content/drive/MyDrive/Zillow_Housing_Data/zhvi_processed_by-zhvi_metro_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zhvi_processed_by-zhvi_state_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zhvi_processed_by-zhvi_county_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zhvi_processed_by-zhvi_city_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zhvi_processed_by-zhvi_zip_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zhvi_processed_by-zhvi_neighborhood_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zori_processed_by-zori_metro_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zori_processed_by-zori_county_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zori_processed_by-zori_city_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zori_processed_by-zori_zip_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zhvf_by-zhvf_metro_cleaned.csv
/content/drive/MyDrive/Zillow_Housing_Data/zhvf_by-zhvf_zip_cleaned.csv
/content/dr

#### Entity Resolution
We can safely assume that the Region IDs used for each locality level are consistent between datasets, as all of these datasets were obtained from the same source.